In [1]:
import pandas as pd
import sys
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_3_nlp import *
from preprocessing.preprocess_2_features import *

In [2]:
data = pd.read_csv('../raw_data/all_game_data_v1_corrected2')

In [136]:
data[data['title'] == 'Starfield']

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
1359,Starfield,2023-09-06,148,343,1.3K,2.9K,"['Bethesda Softworks', 'Bethesda Game Studios']",0.0,"['Adventure', 'RPG', 'Shooter']","['Windows PC', 'Xbox Series']",...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/starfield/


In [3]:
cleaning_function = cleaning_in_notebook(data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [4]:
main_games_cleaned = only_main_games(cleaning_function)

In [5]:
upcoming_mask = main_games_cleaned['release_date'] > pd.Timestamp(datetime.date.today())

In [6]:
released_games = main_games_cleaned[~upcoming_mask]
upcoming_games = main_games_cleaned[upcoming_mask]

In [7]:
released_droped_0_reviews = remove_no_reviews(released_games)

In [8]:
all_relevant_games  =pd.concat((released_droped_0_reviews, upcoming_games))

In [9]:
cols_to_drop = ['ratings_one_zero', 'ratings_one_five',
                    'ratings_two_zero', 'ratings_two_five',
                    'ratings_three_zero', 'ratings_three_five',
                    'ratings_four_zero', 'ratings_four_five',
                    'ratings_five_zero', 'ratings_zero_five',
                    'image', 'main']

In [10]:
all_relevant_games.drop(cols_to_drop, axis=1, inplace=True)

In [11]:
all_relevant_games['description'].fillna('', inplace=True)

In [12]:
all_relevant_games['clean_text'] = all_relevant_games['description'].apply(preprocessing)

In [13]:
vectorizer, vectorizer_sentence = fit_vectorizer(all_relevant_games['clean_text'])

In [14]:
lda = fit_lda(vectorizer_sentence, 25)

In [15]:
all_relevant_games['topics'] = all_relevant_games['clean_text'].apply(nlp_topic, args=(vectorizer, lda))

In [16]:
topics = pd.get_dummies(all_relevant_games['topics'])

In [17]:
topics_data = all_relevant_games.merge(topics, left_index=True, right_index=True)

In [18]:
topics_data.drop(['topics', 'clean_text', 'gen'], axis=1, inplace=True)

In [19]:
df, mlb = categorical_encoder(topics_data['genres'])

In [20]:
genre_cols = keep_x_OHE_columns(df, 12)

In [60]:
genre_cols.isna().sum()

Adventure    0
Indie        0
RPG          0
Simulator    0
Puzzle       0
Shooter      0
Strategy     0
Platform     0
Arcade       0
Sport        0
Racing       0
Fighting     0
dtype: int64

In [61]:
topics_and_genres = topics_data.merge(genre_cols, right_index=True, left_index=True, how='inner')

In [30]:
api_data = pd.read_csv('../raw_data/all_links_with_game_id_v1')
api_data['link'].drop_duplicates(inplace=True)

In [31]:
def clean_urls(url):
    shorter_url = url[25:]
    
    return shorter_url

In [26]:
topics_and_genres['url'] = topics_and_genres['url'].apply(clean_urls)

In [33]:
test = topics_and_genres.merge(api_data, left_on='url', right_on='link', how='left')

In [124]:
topics_and_genres[topics_and_genres['title'] == 'Starfield']

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,RPG,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,Racing,Fighting


In [78]:
links = pd.read_csv('../raw_data/all_links_with_game_id_v1')

In [82]:
dropped_links = links.drop_duplicates()

In [91]:
dropped_links[dropped_links['game_id']==26673]

,link,game_id


In [95]:
links[links['game_id'] == '/games/omori/']

,link,game_id


In [97]:
old_data = pd.read_csv('../raw_data/all_game_data_v1_corrected2')

In [99]:
urls_only = old_data['url'].apply(clean_urls)

In [102]:
url_df = pd.DataFrame(urls_only)

In [107]:
merged_links = links.merge(url_df, left_on='link', right_on='url', how='outer')

In [108]:
merged_links[merged_links['game_id'].isna()]

,link,game_id,url
129007,NaN,NaN,/games/reka/
129008,NaN,NaN,/games/reka/
129009,NaN,NaN,/games/reka/
129010,NaN,NaN,/games/reka/
129011,NaN,NaN,/games/reka/
...,...,...,...
141603,NaN,NaN,/games/taichi-panda--1/
141604,NaN,NaN,/games/silverio-ragnarok/
141605,NaN,NaN,/games/ghostslayer/
141606,NaN,NaN,/games/elifoot/


In [110]:
merged_links[merged_links['url'] == '/games/omori/']

,link,game_id,url
132254,NaN,NaN,/games/omori/


In [119]:
updated = pd.read_csv('../raw_data/updated_all_links')

In [122]:
updated[updated['link'] == '/games/starfield/']

,link,game_id
1044,/games/starfield/,96437
